In [1]:
import numpy as np, healpy as hp, curvedsky, prjlib, plottools as pl, tools_y
from matplotlib.pyplot import *
%matplotlib inline

In [2]:
def show_masks(filename,nmax=5):
    Gmap = {}
    for n in range(nmax):
        Gmap[str(n)] = hp.fitsfunc.read_map(filename,field=n,verbose=False)
        print(np.mean(Gmap[str(n)]))
    pl.view_maps(Gmap,min=0,max=1)

In [4]:
nside = 2048
p  = prjlib.init_analysis()
cy = tools_y.init_compy(p.ids)

### Galactic mask

In [5]:
show_masks(p.fmask_Gfsky,nmax=5)

/global/homes/t/toshiyan/.conda/envs/loc/lib/python3.7/site-packages/healpy/fitsfunc.py:367: UserWarning: No INDXSCHM keyword in header file : assume IMPLICIT
  warnings.warn("No INDXSCHM keyword in header file : " "assume {}".format(schm))


0.20479756593704224
0.40292394161224365
0.600813110669454
0.7004634737968445
0.8000455101331075
map key list: ['0', '1', '2', '3', '4']


### ymap mask

In [6]:
show_masks(cy.fmask_org,nmax=5)

0.5995168577297865
0.499804867503258
0.3989443509380098
0.2993510582951348
0.7600978414217631
map key list: ['0', '1', '2', '3', '4']


In [5]:
ymask = np.zeros((5,12*nside**2))
yptsr = Ymask['4']
for yn in [0,1,2]:
    ymask[yn,:] = yptsr * Ymask[str(yn)]  # add point source mask
ymask[3,:] = ymask[0,:]*tools_y.theta_mask(nside,75.)
ymask[4,:] = ymask[0,:]*tools_y.theta_mask(nside,60.)

0.9659258262890683
0.8660254037844387


In [ ]:
# output custum binary ymask
hp.fitsfunc.write_map(cy.fymask,ymask,overwrite=True) # save

In [6]:
# apodization
def apodize():
    aymask = np.zeros((3,12*nside**2))
    apmask = curvedsky.utils.apodize(nside,yptsr,0.5)
    for n in range(2):
        ymask = hp.fitsfunc.read_map(cy.fmask_org,field=n,verbose=False)
        aymask[n,:] = apmask * curvedsky.utils.apodize(nside,ymask,1.0)
    cmask = tools_y.theta_mask(nside,75.)
    acmask = curvedsky.utils.apodize(nside,cmask,1.0)
    aymask[3,:] = aymask[0,:] * cmask
    cmask = tools_y.theta_mask(nside,60.)
    acmask = curvedsky.utils.apodize(nside,cmask,1.0)
    aymask[4,:] = aymask[0,:] * cmask
    #hp.fitsfunc.write_map(cy.famask,aymask,overwrite=True) # save

0.9659258262890683
0.8660254037844387


In [7]:
show_masks(cy.famask,nmax=5)

0.28710260263692594
0.24515418361443353
0.19925557980751515
0.2792493100582272
0.25561975911475526
map key list: ['0', '1', '2', '3', '4']
